# Register a model in the MLFlow server

We will use the MlflowClient instance to:

- Register a new version for the experiment green-taxi-trip-duration
- Retrieve the latests versions of the model 
- Transition to "Production" and adding annotations to it.

The environment variables are set in the a `.env` file. Create one and add yours.

In [1]:
import mlflow
from mlflow.tracking.client import MlflowClient
import os 
from dotenv import load_dotenv
from datetime import datetime

In [2]:
load_dotenv()
MLFLOW_TRACKING_URI = os.environ.get("MLFLOW_TRACKING_URI")
RUN_ID = os.environ.get("RUN_ID")

Connect to the MLFlow server:

In [3]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

Register the model:

In [4]:
model_uri = f"runs:/{RUN_ID}/model"
mlflow.register_model(model_uri=model_uri, name="yellow-taxi-duration")

Successfully registered model 'yellow-taxi-duration'.
2023/07/16 15:52:24 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: yellow-taxi-duration, version 1
Created version '1' of model 'yellow-taxi-duration'.


<ModelVersion: aliases=[], creation_timestamp=1689515544778, current_stage='None', description='', last_updated_timestamp=1689515544778, name='yellow-taxi-duration', run_id='04f1446bf1344771b953c08328224332', run_link='', source='gs://mlflow-artifacts-2023/taxi/3/04f1446bf1344771b953c08328224332/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

You now also see the model in the MLFlow UI under `Models`.

In [5]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

We can use the MlflowClient to get more information about the model:

In [6]:
model_name = "yellow-taxi-duration"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


We see we only have a version 1 of the model. If we would have registered more artifacts as models under the same name, we would have seen more versions here.
But we want to change the stage from `None` to `Production` and add a description to it. 

First change the stage:

In [7]:
model_version = 1
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1689515544778, current_stage='Production', description='', last_updated_timestamp=1689515571084, name='yellow-taxi-duration', run_id='04f1446bf1344771b953c08328224332', run_link='', source='gs://mlflow-artifacts-2023/taxi/3/04f1446bf1344771b953c08328224332/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

And now add the description:

In [8]:
date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1689515544778, current_stage='Production', description='The model version 1 was transitioned to Production on 2023-07-16', last_updated_timestamp=1689515575488, name='yellow-taxi-duration', run_id='04f1446bf1344771b953c08328224332', run_link='', source='gs://mlflow-artifacts-2023/taxi/3/04f1446bf1344771b953c08328224332/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

If we look at the model again the stage change to Production and the description is added:

In [9]:
latest_versions = client.get_latest_versions(name=model_name)
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}, description: {version.description}")

version: 1, stage: Production, description: The model version 1 was transitioned to Production on 2023-07-16
